# Questions to Address

In [ ]:
d = {} # your dict. What does the f in 'd[f'{file}']' does?
for file in dir_files:
    d[f'{file}']= pd.read_excel(file)

In [ ]:
 ? df.reindex(level=[0,1,2], [range_order, colour_order, fin_order] ) - THIS DOESN'T WORK. Using one line for each level

In [ ]:
 ? ap5a_opex = ap5a.loc[~ap5a['Account'].str[:4].isin ([capex,prod_purchases,winemaking,hospo])]

In [ ]:
# ? Unsure how to call the sheets as dataframes

bs = pd.ExcelFile('Suppliers and Bills Coding.xlsx')

for name in bs.sheet_names:
    df = pd.read_excel(bs, name)
    

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# System Setup

In [ ]:
import pandas as pd
import openpyxl as op
import numpy as np
import glob
import os #e.g. get paths, folders list of files and change active folders
import sys # e.g. get python executable folder
import platform # e.g. information on python version used
import math
from pathlib import Path
import datetime as dt
import re

In [ ]:
# Display more then one output for each cell run (if there is more than one)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Display more columns
pd.options.display.max_columns = 50

# Display format
pd.set_option('display.float.format', '{:,.2f}'.format)

# Cash Flow Reports (AR, AP)

## Dataset preparation

##### Original reports exported from NetSuite

In [ ]:
os.chdir(r'P:\Finance\0-Routine Activities\1-Cashflow management\Reports\Process files')

In [ ]:
ap = pd.read_excel('Cash Flow DB - AP - 2020-10-12.xlsx')

In [ ]:
ar = pd.read_excel('Invoices for cash flow.xlsx')

##### Filling empty line item cells for Due Date and Name (supplier)

In [ ]:
ap1=ap.copy()

In [ ]:
ap1 = ap1.rename(columns = {'Amount.1':'Installment Amount', 'Status.1':"Installment Status"})
ap1['Department'] = ap1['Department'].fillna(method='bfill')
ap1['Name'] = ap1['Name'].fillna(method='ffill')
ap1['Due Date/Receive By'] = ap1['Due Date/Receive By'].fillna(method='ffill')
ap1['Installment Number'] = ap1['Installment Number'].fillna(0)
ap1['Due Date'] = ap1['Due Date'].fillna(ap1['Due Date/Receive By'])
payment_term = (ap1['Due Date/Receive By'] - ap1['Date']).dt.days
entered_to_due = (ap1['Due Date/Receive By'] - ap1['Date Created']).dt.days
ap1.insert(5,'Payment Term', payment_term)
ap1.insert(6,'Entered to Due', entered_to_due)
ap1.insert(21,'Payment Amount', np.where(ap1['Installment Number'] != 0, ap1['Installment Amount'], ap1['Amount']))

In [ ]:
ap1.head()

##### Open BILLS with due date after a given date (e.g. 2 months ago)

In [ ]:
# MAIN LINE
ap1a = ap1[(ap1['Status']=='Open') & (ap1['Due Date/Receive By']>='2020-05-01') & (ap1['*']=='*') & (ap1['Installment Status'] != 'Paid')]

# lINE ITEMS
ap1b = ap1[(ap1['Status']=='Open') & (ap1['Due Date/Receive By']>='2020-05-01') & (ap1['*']!='*')]

# td Create workflow to alert me anytime a bill over $20k is entered in the NetSuite

##### Open INVOICES with due date after a given date (e.g. 2 months ago)

In [ ]:
# MAIN LINE
ar1a = ar[(ar['Status']=='Open') & (ar['Due Date/Receive By']>='2020-05-01') & (ar['*']=='*')]

# LINE ITEMS
ar1b = ar[(ar['Status']=='Open') & (ar['Due Date/Receive By']>='2020-05-01') & (ar['*']!='*')]

## Summary Report

In [ ]:
# Profile
'''
Number of bills, suppliers and amount by department and total
Histogram of bills by amount bins

'''
# Exception Reports
 '''
 BILLS
 No department
 No GL account
 GL account different than default
 Department different than default
 Created less than 10 days before due
 Created more than 5 days after issued
 Payment terms less than 15 days
 
 SUPPLIERS
 No default department
 No default GL account
 Suppliers with more than one department
 Suppliers with more than one GL account
 
 '''

In [ ]:
ap1b.columns

In [ ]:
t1 = pd.DataFrame(ap1b.groupby(['Name'])['Account'].nunique())
t1 = t1.sort_values('Account', ascending=False)
t1 = t1.loc[t1.Account>1]

In [ ]:
t1

### Bills exception reports: no department, matching with default coding (GL account and Department)

In [ ]:
os.chdir(r'P:\Finance\2-Projects\Finance Processes Improvement\Financial Infrastructure')

In [ ]:
bs = pd.read_excel('Suppliers and Bills Coding.xlsx', sheet_name = ['MD_Export', 'MD_RT', 'Bills_Export'])

ns_md = bs['MD_Export']
ws_rt = bs['MD_RT']
bills = bs['Bills_Export']

In [ ]:
bills['Department'] = bills['Department'].fillna(method='bfill', limit=1)
bills['Name'] = bills['Name'].fillna(method='ffill')
bills['Due Date/Receive By'] = bills['Due Date/Receive By'].fillna(method='ffill')

#### No department

In [ ]:
no_dept = bills.loc[(bills.Department.isna()) & (bills['*'] == '*') & (bills['Date Created'].dt.year == 2020)]

#### Multi department and multi GL account

In [ ]:
bills_lines_to_match = bills.loc[(bills['Department'].notna()) \
                                 & (~bills['Name'].str.contains('Inland')) \
                                 & (bills['*'] != '*') \
                                 & (bills['Date Created'] >= '2020-07-01')]

In [ ]:
bills_lines_to_match = bills_lines_to_match.drop_duplicates(['Name','Document Number','Account', 'Department'])

In [ ]:
bills_lines_to_match.to_excel('Bills lines to match text.xlsx')

Multi department

In [ ]:
multi_dept1 = bills_lines_to_match.groupby(['Name'])[['Department']].nunique()
multi_dept1 = multi_dept1.loc[multi_dept1['Department'] >1].sort_values('Department', ascending=False)

In [ ]:
multi_dept_aux = bills_lines_to_match.set_index(bills_lines_to_match['Name']).drop(columns = 'Name')
multi_dept_aux = multi_dept_aux.loc[(multi_dept1.index),:]

In [ ]:
multi_dept2 = multi_dept_aux.groupby(['Name','Department'])[['Date Created']].agg(['max','count'])
multi_dept2[('Date Created','max')] = multi_dept2[('Date Created','max')].dt.strftime('%d-%b-%y')

Multi GL account

In [ ]:
multi_gl1 = bills_lines_to_match.groupby(['Name'])[['Account']].nunique()
multi_gl1 = multi_gl1.loc[multi_gl1['Account'] >1].sort_values('Account', ascending=False)

In [ ]:
multi_gl_aux = bills_lines_to_match.set_index(bills_lines_to_match['Name']).drop(columns = 'Name')
multi_gl_aux = multi_gl_aux.loc[(multi_gl1.index),:]

In [ ]:
multi_gl2 = multi_gl_aux.groupby(['Name','Account']).agg({'Date Created':['max','count'], 'Document Number' : 'max'})
multi_gl2[('Date Created','max')] = multi_gl2[('Date Created','max')].dt.strftime('%d-%b-%y')
multi_gl2.columns = [('Last Document Created On'), ('Document Count'), ('Sample Document')]

## Detailed Views

### Bills

#### Open bills - Supplier View

##### by department

In [ ]:
ap2a = ap1a.pivot_table(index=['Department'], columns=['Due Date/Receive By'], values='Amount', aggfunc='sum')
ap2a = ap2a.resample('M', axis=1).sum()
ap2a.columns = ap2a.columns.strftime('%b-%y')
ap2a['Total'] = ap2a.sum(axis=1)
ap2a.loc['TOTAL'] = ap2a.sum(axis=0)
ap2a = ap2a.sort_values(by=['Department','Total'], ascending=[True,False])
ap2a_name = 'ap_op_grp_by_dept'

##### by department/supplier

In [ ]:
ap2b = ap1a
ap2b = ap2b.pivot_table(index=['Department','Name'], columns='Due Date/Receive By', values='Amount', aggfunc='sum')
ap2b = ap2b.resample('M', axis=1).sum()
ap2b.columns = ap2b.columns.strftime('%b-%y')
ap2b['Total'] = ap2b.sum(axis=1)
ap2b = ap2b.sort_values(by=['Department','Total'], ascending=[True,False])
ap2b_name = 'ap_op_grp_dept_suppl'

##### by department/supplier/doc

In [ ]:
ap2c = ap1a
ap2c = ap2c.drop(columns=['Internal ID','Type','Account','Status'])
ap2c['Date Created'] = ap2c['Date Created'].dt.strftime('%d-%b-%y')
ap2c['Date'] = ap2c['Date'].dt.strftime('%d-%b-%y')
ap2c['Due Date/Receive By'] = ap2c['Due Date/Receive By'].dt.strftime('%d-%b-%y')
ap2c['Due Date'] = ap2c['Due Date'].dt.strftime('%d-%b-%y')
ap2c.set_index(['Department','Name', 'Document Number'],inplace=True)
ap2c = ap2c.sort_index(ascending=[True,True,False])
ap2c_name = 'ap_op_lst_dept_suppl'

##### by due date/supplier/doc

In [ ]:
ap2d = ap1a
ap2d = ap2d.drop(columns=['Internal ID','Type','Account','Status'])
ap2d = ap2d.groupby(['Due Date','Name', 'Document Number', 'Installment Number'])[['Payment Amount']].sum()
ap2d['Cumulative Amount'] = ap2d['Payment Amount'].cumsum()
#ap2d.reset_index(level = [1,2,3], inplace=True)
#ap2d['Due Date'] = ap2d['Due Date'].dt.strftime('%d-%b-%y')
#ap2d = ap2d.drop(columns='Due Date/Receive By')
ap2d_name = 'ap_op_lst_duedat_suppl'

In [ ]:
ap2d.head(30)

#### Open bills - Project View

##### by project

In [ ]:
ap3a = ap1a[ap1a['Project Reference'].notna()]
ap3a = ap3a.groupby(['Project Reference','Due Date/Receive By'])[['Amount']].sum()
ap3a = ap3a.unstack(1)
ap3a = ap3a.resample('M', level=1, axis=1).sum()
ap3a = ap3a.groupby(['Project Reference']).sum()
ap3a.columns = ap3a.columns.strftime('%b-%y')
ap3a_name = 'ap_op_grp_proj'

##### by project/supplier

In [ ]:
ap3b = ap1a[ap1a['Project Reference'].notna()]
ap3b = ap3b.groupby(['Project Reference','Name','Due Date/Receive By'])[['Amount']].sum()
ap3b = ap3b.unstack(2)
ap3b = ap3b.resample('M', level=1, axis=1).sum()
ap3b.columns = ap3b.columns.strftime('%b-%y')
ap3b_name = 'ap_op_grp_proj_suppl'

##### by project/supplier/doc

In [ ]:
ap3c = ap1a[ap1a['Project Reference'].notna()]
ap3c = ap3c.groupby(['Project Reference','Name','Document Number','Due Date/Receive By'])[['Amount']].sum()
ap3c.reset_index(level = 3, inplace = True)
ap3c['Due Date/Receive By'] = ap3c['Due Date/Receive By'].dt.strftime('%d-%b-%y')
ap3c_name = 'ap_op_lst_proj_suppl'

##### Weekly payments schedule

In [ ]:
ap4a = ap1a
ap4a = ap4a.set_index('Due Date/Receive By')
ap4a = ap4a.resample('W-SUN', axis=0)['Amount'].sum().reset_index()
ap4a = ap4a.rename(columns = {'Due Date/Receive By' : 'Pmt Week Ending'})
ap4a = ap4a.set_index('Pmt Week Ending')
ap4a_name = 'ap_op_weekly_amt'

#### Open bills - Category View

In [ ]:
temp_lst

In [ ]:
temp_lst = pd.DataFrame(ap1b['Account'].unique(), columns = ['Account'])
temp_lst['Code'] = temp_lst.Account.str[:4]
temp_lst.sort_values('Code')

In [ ]:
# Auxiliary dataframe for category reporting

ap5a = ap1b
capex = ['5615', '6020']
winemaking = ['5523', '5525']
prod_purchases = ['3004', '5532', '4045', '2099']
hospo = ['2044']
ap5a_capex = ap5a.loc[ap5a['Account'].str[:4].isin(capex)]
ap5a_prod = ap5a.loc[(ap5a['Account'].str[:4].isin (prod_purchases))]
ap5a_winemaking = ap5a.loc[(ap5a['Account'].str[:4].isin (winemaking))]
ap5a_hospo = ap5a.loc[(ap5a['Account'].str[:4].isin (hospo))]
# ? ap5a_opex = ap5a.loc[~ap5a['Account'].str[:4].isin ([capex,prod_purchases,winemaking,hospo])]
ap5a_aggreg_lst = [ap5a_capex, ap5a_prod, ap5a_winemaking, ap5a_hospo]
ap5a_aggreg = pd.concat(ap5a_aggreg_lst, keys = ['Capex', 'Production', 'Winemaking', 'Hospitality'])
ap5a_aggreg.index.names = ['Group', 'ID']

In [ ]:
# Weekly bills by category (capex, opex, production, winemaking, hospo)

ap5b = ap5a_aggreg.groupby(['Group', 'Due Date/Receive By'])[['Amount']].sum()
ap5b = ap5b.unstack(1)
ap5b = ap5b.resample('W-SUN', level=1, axis=1).sum()
ap5b.loc['TOTAL'] = ap5b.sum(axis=0)

In [ ]:
# Monthly bills by capex supplier

ap5c = ap5a_aggreg.groupby(['Group', 'Name','Due Date/Receive By'])[['Amount']].sum()
ap5c = ap5c.reset_index()
ap5c = ap5c.loc[ap5c['Group'] == 'Capex']
ap5c.set_index(['Group','Name','Due Date/Receive By'], inplace=True)
ap5c = ap5c.unstack(2)
ap5c = ap5c.resample('M', level=1, axis=1).sum()

In [ ]:
ap5b.head()

## Invoices

### Open invoices schedule

#### by department

In [ ]:
ar2a = ar1a.pivot_table(index=['Department'], columns=['Due Date/Receive By'], values='Amount', aggfunc='sum')
ar2a = ar2a.resample('M', axis=1).sum()
ar2a.columns = ar2a.columns.strftime('%b-%y')
ar2a['Total'] = ar2a.sum(axis=1)
ar2a.loc['TOTAL'] = ar2a.sum(axis=0)
ar2a = ar2a.sort_values(by=['Department','Total'], ascending=[True,False])
ar2a_name = 'ar_op_grp_by_dept'

#### by department/supplier

In [ ]:
ar2b = ar1a
ar2b = ar2b.pivot_table(index=['Department','Name'], columns='Due Date/Receive By', values='Amount', aggfunc='sum')
ar2b = ar2b.resample('M', axis=1).sum()
ar2b.columns = ar2b.columns.strftime('%b-%y')
ar2b['Total'] = ar2b.sum(axis=1)
ar2b = ar2b.sort_values(by=['Department','Total'], ascending=[True,False])
ar2b_name = 'ar_op_grp_dept_suppl'

#### by department/supplier/doc

In [ ]:
ar2c = ar1a
ar2c = ar2c.drop(columns=['Internal ID','Type','Account','Status'])
ar2c['Date Created'] = ar2c['Date Created'].dt.strftime('%d-%b-%y')
ar2c['Date'] = ar2c['Date'].dt.strftime('%d-%b-%y')
ar2c['Due Date/Receive By'] = ar2c['Due Date/Receive By'].dt.strftime('%d-%b-%y')
ar2c.set_index(['Department','Name', 'Document Number'],inplace=True)
ar2c = ar2c.sort_index(ascending=[True,True,False])
ar2c_name = 'ar_op_lst_dept_suppl'

#### by due date/supplier/doc

In [ ]:
ar2d = ar1a
ar2d = ar2d.drop(columns=['Internal ID','Type','Account','Status'])
ar2d['Due Date'] = ar2d['Due Date/Receive By'].dt.strftime('%d-%b-%y')
ar2d = ar2d.groupby(['Due Date/Receive By','Due Date','Name', 'Document Number'])[['Amount']].sum()
ar2d.reset_index(level=0, inplace=True)
ar2d = ar2d.drop(columns='Due Date/Receive By')
ar2d_name = 'ar_op_lst_duedat_suppl'

#### Weekly payments schedule

In [ ]:
ar4a = ar1a
ar4a = ar4a.set_index('Due Date/Receive By')
ar4a = ar4a.resample('W-SUN', axis=0)['Amount'].sum().reset_index()
ar4a = ar4a.rename(columns = {'Due Date/Receive By' : 'Pmt Week Ending'})
ar4a = ar4a.set_index('Pmt Week Ending')
ar4a_name = 'ar_op_weekly_amt'

## Export to Excel

In [ ]:
def ap_output_to_excel_multisheets(reports_list, sheets_list, file_name):
    writer = pd.ExcelWriter(file_name)
    n=0
    for report, sheet in zip(reports_list, sheets_list):
        report.to_excel(writer, sheet_name = sheet, startrow = 0, startcol = 0)
        n+=1
    writer.save()

reports = [ap1, ap2a, ap2b, ap2c, ap2d, ap3a, ap3b, ap3c, ap4a]
sheets = ['Export AP DB', ap2a_name, ap2b_name, ap2c_name, ap2d_name, ap3a_name, ap3b_name, ap3c_name, ap4a_name]

ap_output_to_excel_multisheets(reports, sheets, 'Cash Flow reports - AP - 2020-10-12.xlsx')

In [ ]:
def ar_output_to_excel_multisheets(reports_list, sheets_list, file_name):
    writer = pd.ExcelWriter(file_name)
    n=0
    for report, sheet in zip(reports_list, sheets_list):
        report.to_excel(writer, sheet_name = sheet, startrow = 0, startcol = 0)
        n+=1
    writer.save()

reports = [ar2a, ar2b, ar2c, ar2d, ar4a]
sheets = [ar2a_name, ar2b_name, ar2c_name, ar2d_name, ar4a_name]

ar_output_to_excel_multisheets(reports, sheets, 'Cash Flow reports - AR.xlsx')

# Beverage Sales Reports

In [ ]:
os.chdir(r'C:\Users\joao.melo\Sutton Group\Budget FY21 - Documents\General\03 - Current Version')

### Preparing the database and aggregated results

Reading the database and excluding unnecessary data

In [ ]:
a = pd.read_excel('Consolidated Model-v4.xlsx', skiprows=3, usecols = 'A:BS', sheet_name='Sales Fcst')
# Verify if columns (usecols) appropriate for every new file

In [ ]:
b=a.drop(columns = ['Channel 3', 'Activity']) 

Melting the file into database format and adding required financial information (revenues, COGS, FY)

In [ ]:
c=b.melt(['Range','Colour','Varietal','Market','Channel 1','Channel 2','Unit COGS',\
          'Unit Price'],var_name='Period',value_name='Units',)

In [ ]:
c['Revenue'] = c['Unit Price'] * c['Units']
c['COGS'] = c['Unit COGS'] * c['Units']
c['Margin'] = c['Revenue'] - c['COGS']

In [ ]:
fy20 = c.loc[(c.Period>='2019-08-01') & (c.Period<='2020-07-01'),:]
fy21 = c.loc[(c.Period>='2020-08-01') & (c.Period<='2021-07-01'),:]
fy22 = c.loc[(c.Period>='2021-08-01') & (c.Period<='2022-07-01'),:]
fy23 = c.loc[(c.Period>='2022-08-01') & (c.Period<='2023-07-01'),:]

fin_years = [fy20, fy21, fy22, fy23]

c = pd.concat(fin_years, keys=['FY20', 'FY21', 'FY22', 'FY23'])

c = c.reset_index().drop(columns='level_1').rename(columns={'level_0':'FY'})

In [ ]:
print('Ranges are ',c['Range'].unique())
print('Varietals are ',c['Varietal'].unique())
print('Markets are ',c['Market'].unique())
print('Channels 1 are ',c['Channel 1'].unique())
print('Channels 2 are ',c['Channel 2'].unique())

#### Deprecated code that was used to verify data consistency and add Colour field
Testing and fixing inconsistent data

Verify if ranges listed below match all the ones in source file

whites_lst = ['Sauv Blanc', 'White Mischief', 'Chardonnay', 'Pinot Gris', 'Rose', 'Albarino',\
              'Cider', 'Sauv Blanc - 12x375ml', 'Water']
reds_lst = ['Merlot Malbec', 'Malbec Merlot', 'Red', 'Pinot Noir', 'Tennessee Red', 'Lagrein',\
            'Syrah', 'Red - 12x375ml']

c['Range']=c['Range'].str.strip()

c['Channel 1'] = c['Channel 1'].replace('NZ - Off site', 'NZ - Off Site')

c.loc[pd.isnull(c['Channel 1']),'Channel 1'] = c['Market']

c.loc[pd.isnull(c['Channel 2']),'Channel 2'] = c['Market']

if len(c['Range'].unique())>8:
    print('Verify possible typos/duplication/NaN in Range data')
if len(c['Varietal'].unique())> (len(whites_lst) + len(reds_lst)):
    print('Verify possible typos/duplication/NaN in Varietal data')
if len(c['Market'].unique())>4:
    print('Verify possible typos/duplication/NaN in Market data')
if len(c['Channel 1'].unique())>5:
    print('Verify possible typos/duplication/NaN in Channel 1 data')
if len(c['Channel 2'].unique())>7:
    print('Verify possible typos/duplication/NaN in Channel 2 data')

Adding required colour information (reds, whites)

reds = c.loc[c['Varietal'].isin(reds_lst)]
whites = c.loc[c['Varietal'].isin(whites_lst)]
colour = [whites, reds]

c = pd.concat(colour, keys=['White','Red'])
c = c.reset_index().drop(columns='level_1').rename(columns={'level_0':'Colour'})

In [ ]:
temp_df = c.loc[c['FY'] == 'FY21']
temp_df

In [ ]:
temp_df.columns

In [ ]:
margin_by_channel = temp_df.groupby(['Range','Channel 2'])[['Revenue','Margin', 'Units']].sum()
margin_by_channel

In [ ]:
margin_by_channel['Gross Margin'] = margin_by_channel['Margin'] / margin_by_channel['Revenue']

In [ ]:
margin_by_channel = margin_by_channel[['Gross Margin', 'Units']].dropna()

In [ ]:
margin_by_channel = margin_by_channel.drop(index = ['Homeblock', 'Project X', 'Collectors', 'Blush Crush'], level=0)

In [ ]:
margin_by_channel = margin_by_channel.sort_index(ascending=True, level=0)

In [ ]:
margin_by_channel.to_excel('Margin by Channel.xlsx')

## Channel views

Monthly report by Segment

In [ ]:
total = c
total.name = 'TOTAL'

In [ ]:
total_excl_onsite = c.loc[c['Channel 1'] != 'NZ - On Site']
total_excl_onsite.name = 'TOTAL EXCL ONSITE'

In [ ]:
domestic = c.loc[c['Channel 1'] == 'NZ - Off Site']
domestic.name = 'DOMESTIC'

In [ ]:
export = c.loc[(c['Market'] != 'NZ') & (c['Market'] != 'China')]
export.name = 'EXPORTS'

In [ ]:
china = c.loc[c['Market'] == 'China']
china.name = 'CHINA'

In [ ]:
onsite = c.loc[c['Channel 1'] == 'NZ - On Site']
onsite.name = 'ON SITE'

### Output Tables

Export Monthly report by Segment

In [ ]:
def segments_monthly_to_excel_singlesheet(views, sheets, file_name):
    writer = pd.ExcelWriter(file_name)
    row = 2
    
    for view, name in zip(views, names):
        
        view = view.groupby(['Period'])[['Units','Revenue','COGS','Margin']].sum()
        view = view.T
        
        fin_order = ['Units', 'Revenue', 'COGS', 'Margin']
        view = view.reindex(fin_order)
        
        view.to_excel(writer, sheet_name = sheets, startrow = row, index_label = name, startcol = 0, float_format = '%.0f')
        
        row = row + len(view.index) + 3
    
    writer.save()


views = [total, total_excl_onsite, domestic, export, china, onsite]
names = ['TOTAL', 'TOTAL EXCL ONSITE', 'DOMESTIC', 'EXPORTS', 'CHINA', 'ON SITE'] # ? why not accepting view.name in index_label?
sheets = 'Monthly'
segments_monthly_to_excel_singlesheet(views, sheets, 'Beverage Sales Report Monthly.xlsx')  

Export Monthly report by Segment/Range/Colour

In [ ]:
def segments_by_range_colour_to_excel_singlesheet(views, sheets, file_name):
    writer = pd.ExcelWriter(file_name)
    row = 2
    for view in views:
        
        view = view.groupby(['Range', 'Colour', 'Period'])[['Units','Revenue','COGS','Margin']].sum()
        view = view.unstack(2).stack(0)
        
        range_order = ['Expressions', 'Seasonal', 'Homeblock', 'Collectors', 'Blush Crush', 'Project X', 'X']
        colour_order = ['White & Rose', 'Red']
        fin_order = ['Units', 'Revenue', 'COGS', 'Margin']
        view = view.reindex(range_order, level=0)
        view = view.reindex(colour_order, level=1)
        view = view.reindex(fin_order, level=2)
        view.columns = view.columns.strftime('%b-%y')
        
        view.to_excel(writer, sheet_name = sheets, startrow = row, startcol = 0, float_format = '%.0f')
        
        row = row + len(view.index) + 3
    
    writer.save()

views = [total, total_excl_onsite, domestic, export, china, onsite]
sheets = 'Monthly'
segments_by_range_colour_to_excel_singlesheet(views, sheets, 'Beverage Sales Report by Range-Colour.xlsx')  

### By Market

In [ ]:
volumes_by_market = c.groupby(['FY','Market'])['Units'].sum()

volumes_by_market = volumes_by_market.unstack(0).stack().unstack(0)

volumes_by_market['Total'] = volumes_by_market.sum(axis=1, skipna=True)

revenues_by_market = c.groupby(['FY','Market'])['Revenue'].sum()

revenues_by_market = revenues_by_market.unstack(0).stack().unstack(0)

revenues_by_market['Total'] = revenues_by_market.sum(axis=1, skipna=True)

margin_by_market = c.groupby(['FY','Market'])['Margin'].sum()

margin_by_market = margin_by_market.unstack(0).stack().unstack(0)

margin_by_market['Total'] = margin_by_market.sum(axis=1, skipna=True)

### By Product

In [ ]:
volumes_by_product = c.groupby(['FY','Range', 'Varietal'])['Units'].sum()

volumes_by_product = volumes_by_product.unstack(0).stack().unstack(0)

volumes_by_product['Total'] = volumes_by_product.sum(axis=1, skipna=True)

revenues_by_product = c.groupby(['FY','Range', 'Varietal'])['Revenue'].sum()

revenues_by_product = revenues_by_product.unstack(0).stack().unstack(0)

revenues_by_product['Total'] = revenues_by_product.sum(axis=1, skipna=True)

margin_by_product = c.groupby(['FY','Range', 'Varietal'])['Margin'].sum()

margin_by_product = margin_by_product.unstack(0).stack().unstack(0)

margin_by_product['Total'] = margin_by_product.sum(axis=1, skipna=True)

# Brix China Financials

In [ ]:
os.chdir(r'P:\Finance\6-Brix China\Financials')

In [ ]:
dir_files = list(glob.glob('*P&L.xlsx'))

In [ ]:
all_month_rep = [m for m in dir_files]
reps = [pd.read_excel(month_rep, skiprows=2, nrows=16) for month_rep in all_month_rep]
df_names=[]
for month_rep in all_month_rep:
    name=month_rep[:3]
    df_names.append(name)

a=zip(reps,df_names)

for rep,df_name in a:
    rep['Source']=df_name

In [ ]:
def mult_reps_mult_sheets(reports, file_name):
    writer = pd.ExcelWriter(file_name)
    n=0
    for report in reports:
        report.to_excel(writer, sheet_name = str(df_names[n]), startrow = 0, startcol = 0)
        n+=1
    writer.save()

mult_reps_mult_sheets(reps, 'Output.xlsx')

In [ ]:
def mult_reps_single_sheet(reports, file_name):
    writer = pd.ExcelWriter(file_name)
    n=0
    row=1
    for report in reports:
        report.to_excel(writer, sheet_name = 'Single Sheet', startrow = row, startcol = 0)
        n+=1
        row=row+len(report)+3
    writer.save()

mult_reps_single_sheet(reps, 'Output.xlsx')

# Brix Revenues, Costs and Production Planning

In [ ]:
import pandas as pd
import openpyxl as op
import numpy as np
import glob
import os
import math
from pathlib import Path
import datetime as dt

In [ ]:
os.chdir(r'C:\Users\joao.melo\Sutton Group\Budget FY21 - Documents\Forecasts\01-Sep')

In [ ]:
src_file = pd.ExcelFile('CoPack - FY21 Budget vs Fcst 09.09.20.xlsx')

### Preparing the volumes database (filtered by 'in budget')

In [ ]:
a1 = pd.read_excel(src_file, skiprows=4, sheet_name='Combined Detail')

In [ ]:
b = a1.drop(columns=['FY20 Val', 'FY21 Val', 'FY22 Val', 'FY23 Val',
       'Aug-20 Val', 'Sep-20 Val', 'Oct-20 Val', 'Nov-20 Val', 'Dec-20 Val',
       'Jan-21 Val', 'Feb-21 Val', 'Mar-21 Val', 'Apr-21 Val', 'May-21 Val',
       'Jun-21 Val', 'Jul-21 Val']).copy()

In [ ]:
b = b.melt(['DEPARTMENT', 'PROD_ID', 'CUST_ID', 'PROD_PLAN_ID', 'PRODUCTION_ID', 'Customer',\
       'Average_Price', 'Product_Category', 'Type', 'Status', 'Included_in_Budget',\
       'Long_Term_Demand', 'Inner_Packaging/Multi label', 'Blending',\
       'Pasteurised', 'Run_Size', 'Vessel_Type', 'Vessel_Size',\
       'Vessel_Vol_(ml)', 'Baseline_Vol', 'Growth_FY1-FY2', 'Growth_FY2-FY3','FY20', 'FY21', 'FY22', 'FY23'], var_name = 'Period', value_name = 'Units', )

In [ ]:
b.Period = '01-' + b.Period.astype(str)

b.Period = pd.to_datetime(b.Period)

In [ ]:
b['Revenue'] = b['Average_Price'] * b['Units']

In [ ]:
fy20 = b.loc[(b.Period>='2019-08-01') & (b.Period<='2020-07-01'),:]
fy21 = b.loc[(b.Period>='2020-08-01') & (b.Period<='2021-07-01'),:]
fy22 = b.loc[(b.Period>='2021-08-01') & (b.Period<='2022-07-01'),:]
fy23 = b.loc[(b.Period>='2022-08-01') & (b.Period<='2023-07-01'),:]

fin_years = [fy20, fy21, fy22, fy23]

b = pd.concat(fin_years, keys=['FY20', 'FY21', 'FY22', 'FY23'])

b = b.reset_index().drop(columns='level_1').rename(columns={'level_0':'FY'})

In [ ]:
b = b[b.Included_in_Budget == 'Yes']
b = b[b.Type == 'Forecast']

### Production configurations and parameters table (excl. non programmed items - e.g. Kegs)

Production Database (e.g. excludes volumes not used in production planning)

In [ ]:
c0 = b.loc[b['PROD_PLAN_ID'] != 'None']

Production Parameters Table

In [ ]:
d0 = pd.read_excel(src_file, skiprows=1, usecols='C:O', nrows=8,  sheet_name='Production - CP')
d0 = d0.dropna(axis=1)
d0 = d0.set_index('Parameter')

In [ ]:
d0

In [ ]:
d0t = d0.T

In [ ]:
d0

In [ ]:
d0t

In [ ]:
'''
d1 = pd.DataFrame({'PROD_PLAN_ID':c0['PROD_PLAN_ID'].unique()})
d1 = d1.set_index('PROD_PLAN_ID')
d1 = d1.sort_values('PROD_PLAN_ID')
'''

Customers volume per configuration

In [ ]:
top5_cust_1 = c0.loc[c0['FY'] == 'FY21']
top5_cust_1 = top5_cust_1.groupby(['PROD_PLAN_ID','Customer'])[['Units']].sum()
top5_cust_1.sort_values(['PROD_PLAN_ID','Units'], ascending=[True,False])

Production Tables

In [ ]:
prod_vols = c0.groupby(['PROD_PLAN_ID', 'FY', 'Period'])[['Units']].sum()
prod_vols = prod_vols.unstack(0)
prod_vols['Period Total'] = prod_vols.sum(axis=1)

In [ ]:
prod_hours = prod_vols.div(d0.iloc[6], level=1, axis=1)
prod_hours_total = prod_hours.sum(axis=1)
prod_hours['Period Total'] = prod_hours.sum(axis=1)

In [ ]:
unit_price = c0.groupby(['PROD_PLAN_ID', 'FY', 'Period'])[['Units', 'Revenue']].sum()
unit_price['Average Price'] = unit_price['Revenue'] / unit_price['Units']
unit_price = unit_price.drop(columns = ['Units', 'Revenue'])
unit_price = unit_price.unstack(0)

avg_unit_price = c0.groupby(['FY', 'Period'])[['Units', 'Revenue']].sum()
avg_unit_price['Avg Unit Price'] = avg_unit_price['Revenue'] / avg_unit_price['Units']
avg_unit_price = avg_unit_price.drop(columns = ['Revenue', 'Units'])

unit_price['Avg Unit Price'] = avg_unit_price['Avg Unit Price']

In [ ]:
manuf_cost = pd.read_excel(src_file, skiprows=1, usecols='D:Y', sheet_name='Manuf Cost')

In [ ]:
unit_cost = prod_hours.div(prod_hours_total, level=1, axis=0) / prod_vols
unit_cost = unit_cost.mul(manuf_cost['TOTAL MANUFACTURING COST'].values, axis=0)
unit_cost = unit_cost.rename(columns = {'Period Total' : 'Avg Unit Cost'})

In [ ]:
unit_dir_cost = prod_hours.div(prod_hours_total, level=1, axis=0) / prod_vols
unit_dir_cost = unit_dir_cost.mul(manuf_cost['TOTAL DIRECT LABOUR COST'].values, axis=0)
unit_dir_cost = unit_dir_cost.rename(columns = {'Period Total' : 'Avg Unit Cost'})

In [ ]:
unit_margin = unit_price.sub(unit_cost.values, level=1)

In [ ]:
unit_dir_margin = unit_price.sub(unit_dir_cost.values, level=1)

In [ ]:
nom_margin = unit_margin.mul(prod_vols.values)
nom_margin['Period Total'] = nom_margin.sum(axis=1)

In [ ]:
# avg unit direct margin per FY

aum_lst = []
for fy in ['FY20', 'FY21', 'FY22', 'FY23']:
    m = ((unit_dir_margin['Avg Unit Price'].loc[fy].mul(prod_vols['Period Total'].loc[fy])).sum())/(prod_vols['Period Total'].loc[fy].sum())
    aum_lst.append(m)

### Sales Volumes, Revenues and Avg Price per product per month DB

Excluding kegs and Brix volumes and None IDs

In [ ]:
c1 = b
c1 = c1[~c1['PROD_PLAN_ID'].isin(['WNKL','None'])]
c1 = c1[c1['CUST_ID'] != 'BRIED']

Revenues by Period (Excludes kegs and Brix volumes)

In [ ]:
c2a = c1
c2a = c2a.groupby(['Period'])[['Revenue']].sum()
c2a.index = prod_vols.index
revenue_by_period = c2a

Revenues by Production Plan ID

In [ ]:
c2b = c1
c2b = c2b.groupby(['FY','Period','PROD_PLAN_ID'])[['Revenue']].sum()
c2b = c2b.unstack(2)
revenue_by_prod_plan_id = c2b

Annual figures

In [ ]:
annual_vols = prod_vols.loc['FY21']
annual_vols = annual_vols.sum()

In [ ]:
annual_prod_hours = prod_hours.loc['FY21'].sum()

In [ ]:
annual_revenues = revenue_by_prod_plan_id.loc['FY21'].sum()
annual_revenues['Period Total'] = annual_revenues.sum()

In [ ]:
annual_dir_labour_cost = unit_dir_cost.loc['FY21'].sum()

Production Analysis Reports

In [ ]:
d1 = d0t
d1['Production Hours FY21'] = p1
d1['Production Volume FY21'] = ''
d1['Revenues FY21'] = ''
d1['Avg Price FY21'] = ''
d1['Revenue per Hour FY21'] = ''
d1['Direct Margin per Hour FY21'] = ''
d1['Total Direct Margin FY21'] = ''


In [ ]:
d1 = d0t

In [ ]:
p1 = prod_hours.loc['FY21'].drop(columns='Period Total').sum()

In [ ]:
p2 = prod_vols.loc['FY21'].drop(columns='Period Total').sum()

In [ ]:
p3 = revenue_by_prod_plan_id.loc['FY21'].sum()
p3.index = p2.index

In [ ]:
p4 = ''

In [ ]:
p5 = p3/p1
p5.index = p2.index

In [ ]:
p6 = 

In [ ]:
data = {'Production Hours FY21' : p1, 'Production Volume FY21' : p2, \
        'Revenues FY21' : p3, 'Avg Price FY21' : p4, 'Revenue per Hour FY21' : p5, \
       'Direct Margin per Hour FY21' : p6, 'Total Direct Margin FY21' : p7}

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df

In [ ]:
d1

In [ ]:
# Production Report - Parameters per Hour



In [ ]:
# Production Report - Parameters per Unit


In [ ]:
# Production Report - Production Configuration Analysis


### Dry Goods Sales

In [ ]:
dgdb = pd.read_excel('Dry Goods Parameters.xlsx')

In [ ]:
dg_items_cust.to_excel('DG Items by Customer.xlsx')

In [ ]:
dg_items_cust = dgdb.groupby(['CUST_ID','Customer','Item'])['DG_Unit_Price'].sum()

In [ ]:
dg_unit_price_cust = dgdb.groupby(['CUST_ID','Customer'])['DG_Unit_Price'].sum()

In [ ]:
dg_rev_db = pd.merge(b,dg_unit_price_cust, on='CUST_ID', how='left')

In [ ]:
dg_rev_db['DG Revenue'] = dg_rev_db['Units'] * dg_rev_db['DG_Unit_Price']

In [ ]:
dg_rev = dg_rev_db.groupby(['Customer'])['DG Revenue'].sum()

In [ ]:
dg_rev_db.to_excel('DG Revenues.xlsx')

### Output to Excel

In [ ]:
def output_to_excel_multisheets(reports_list, sheets_list, file_name):
    writer = pd.ExcelWriter(file_name)
    n=0
    for report, sheet in zip(reports_list, sheets_list):
        report.to_excel(writer, sheet_name = sheet, startrow = 0, startcol = 0)
        n+=1
    writer.save()

reports = [production_parameters, prod_vols, prod_hours, unit_price, manuf_cost, unit_cost, unit_margin, nom_margin, revenues]
sheets = ['production_parameters', 'prod_vols', 'prod_hours', 'unit_price', 'manuf_cost', 'unit_cost', 'unit_margin', 'nom_margin', 'revenues']

output_to_excel_multisheets(reports, sheets, 'Sales and Production Planning- v4.2.xlsx')


# Credit Cards

In [ ]:
import pandas as pd
import openpyxl as op
import numpy as np
import datetime as dt
import glob
import os
import smtplib

### Card users list

pd.read_csv , map , format

In [ ]:
card_users = pd.read_csv ('card_users.csv') # td Adapt to read csv from proper file location

In [ ]:
card_users['Card_ID'] = card_users['Card_ID'].map('{:04.0f}'.format)

### Bank statement treatment

glob, df.drop, applying math formula, df.loc, input, pd.to_datetime, datetime.strftime, df.str

In [ ]:
for file in glob.glob('TransHist*.csv'):
    bank_statement = pd.read_csv (file) # td Adapt to read csv without specific name of file

In [ ]:
bank_statement = bank_statement.drop(bank_statement.columns[[7]], axis = 1)

In [ ]:
bank_statement.Amount = bank_statement.Amount * -1

In [ ]:
bank_statement = bank_statement.loc[bank_statement['Tran Type'] != 'PAY', :]

In [ ]:
Drop_Date = input('Type date to exclude') # td Add possibility to not exclude any date or to exclude more than one date

In [ ]:
bank_statement = bank_statement.loc[bank_statement['Date'] != Drop_Date, :]

In [ ]:
print(bank_statement.Amount.sum()) # td Add ability to capture target amount and match csv file sum with it + Format 2 decimals

In [ ]:
bank_statement['Reference'] = bank_statement['Reference'].map('{:.0f}'.format)

In [ ]:
bank_statement['Date'] = pd.to_datetime(bank_statement['Date']).dt.strftime('%d-%b-%y')

In [ ]:
bank_statement['Card_ID'] = bank_statement['Reference'].str[-4:]

### Consolidated dataframe

pd.merge, create new column, df.to_excel

In [ ]:
consolid_1 = pd.merge(bank_statement, card_users, how='left', on='Card_ID')

In [ ]:
consolid_1['Transaction_ID'] = consolid_1.index

In [ ]:
consolid_1.to_excel('C:\\Users\\joao.melo\\Downloads\\2020-04\\consolid_1.xlsx')

### Individual reports prep

df.drop_duplicates, df.sort_values, df.reset_index, for loop, op.load_workbook, df.active (sheet), ws.column_dimensions (width), ws.row_dimensions (in for loop)

In [ ]:
Employees = consolid_1['Employee'].drop_duplicates().sort_values().reset_index(drop=True)

In [ ]:
for Employee in Employees:
    individual_report = consolid_1[(consolid_1.Employee == Employee)]
    individual_report = (individual_report[['Date', 'Amount', 'Payee', 'Card_ID']])
    individual_report['Description'] = ''
    individual_report.to_excel(str(Employee) + '-card.xlsx', index_label = 'Transaction_ID')

In [ ]:
# With iter_rows

for Employee in Employees:
    individual_report_xl = op.load_workbook(str(Employee) + '-card.xlsx')
    report_sheet = individual_report_xl.active
    for row in report_sheet.iter_rows(2,None):
        report_sheet.row_dimensions.height=20
    report_sheet.column_dimensions['B'].width = 15
    report_sheet.column_dimensions['C'].width = 15
    report_sheet.column_dimensions['D'].width = 30
    report_sheet.column_dimensions['E'].width = 10
    report_sheet.column_dimensions['F'].width = 45
#    for i in range(0,30):
#        report_sheet.row_dimensions[i+1].height=20
    individual_report_xl.save(str(Employee) + '-card.xlsx')

In [ ]:
# With for loop

for Employee in Employees:
    individual_report_xl = op.load_workbook(str(Employee) + '-card.xlsx')
    report_sheet = individual_report_xl.active
    report_sheet.column_dimensions['B'].width = 15
    report_sheet.column_dimensions['C'].width = 15
    report_sheet.column_dimensions['D'].width = 30
    report_sheet.column_dimensions['E'].width = 10
    report_sheet.column_dimensions['F'].width = 45
    for i in range(0,30):
        report_sheet.row_dimensions[i+1].height=20
    individual_report_xl.save(str(Employee) + '-card.xlsx')

Is there a practical way to autofit column widths as this is quite a long code for not so much value?

### Emailing individual reports to card users

In [ ]:
sender_email = 'joao.melo@brix.co.nz'
rec_email = 'joao.melo@brix.co.nz'
subject = 'Testing Python email'
password = 'TheL0dg3!!'
message = 'Subject: Testing Python email.\nHey, this is an email test using Python'

server = smtplib.SMTP('smtp.office365.com', 587)
type(server)

In [ ]:
server.ehlo()

In [ ]:
server.starttls()

In [ ]:
server.login(sender_email, password)

In [ ]:
print('Login success')
server.sendmail(sender_email, rec_email, message)
print('Email has been sent to ', rec_email)

In [ ]:
server.quit()

### Collating updated individual reports

glob, loop in glob, pd.concat (with loop)

In [ ]:
all_individ_reports = [i for i in glob.glob('*-card.xlsx')]

In [ ]:
combined_individ_reports = pd.concat([pd.read_excel(f) for f in all_individ_reports], ignore_index=True)

In [ ]:
combined_individ_reports.to_excel('Combined_Individual_Reports.xlsx')

### Merging individual report updates with Consolidated dataframe

In [ ]:
consolid_2 = (pd.merge(consolid_1, combined_individ_reports[['Transaction_ID', 'Description']],
            on='Transaction_ID', how = 'left'))

In [ ]:
consolid_2.to_excel('Consolidated_pre_coding.xlsx')

In [ ]:
import pandas as pd
import openpyxl as op
import numpy as np
import datetime as dt
import glob
import os
import smtplib

# Projects Capex Analysis

In [ ]:
a=pd.read_excel('Capex bills.xlsx')

In [ ]:
a.columns

In [ ]:
#a=a.drop(columns='Comments')
a=a.melt(['Suppliers'], var_name='Period', value_name='Amount')

In [ ]:
b=a.loc[a.Period>='2020-06-01']

In [ ]:
c=b.groupby(['Suppliers','Period'])['Amount'].sum().sort_values(ascending=False)

In [ ]:
d=c.unstack('Period')

In [ ]:
d['Total']=d.loc[:,'2020-06-01':'2020-09-01'].sum(1)

In [ ]:
e=d.sort_values('Total', ascending=False)

In [ ]:
e.to_excel('Capex bills out.xlsx')

# Storage Charges - DGs and FGs

In [ ]:
os.chdir(r'P:\Finance\0-Routine Activities\0-Invoicing\01 Aug 2020\Storage\TWE')

In [ ]:
stor_a = pd.read_excel('TWE Storage 29072020-01092020.xlsx', skiprows = 6, sheet_name = ['W1','W2','W3','W4','W5','Excluded Stock','Pallet data'])

##### TWE

Capture information from reports extracted into spreadsheets

In [ ]:
w1 = stor_a['W1']
w2 = stor_a['W2']
w3 = stor_a['W3']
w4 = stor_a['W4']
w5 = stor_a['W5']
excl = stor_a['Excluded Stock']
pallet_conv = stor_a['Pallet data']

weeks = [w1,w2,w3,w4,w5]

w_all = pd.concat(weeks, copy=True)

Prepare relevant items and volumes to be charged

In [ ]:
weeks = [w[(w['Item'].str[3] != 'L') & (w['Item'].str[3] != 'P') & \
           (w['Product Owner'] == 'CUS000404 Treasury Wine Estates') \
           & (w['Ending Inv Qty On-hand'] != 0)] for w in weeks]

In [ ]:
weeks = [w.fillna(0) for w in weeks]

In [ ]:
weeks = [w.groupby(['Week','Item Type: Long Name','Item','Display Name','Units per Pallet','Pallet Type']) \
         [['Ending Inv Qty On-hand']].mean().reset_index() for w in weeks]

In [ ]:
twe_w_all = pd.concat(weeks)

Create list of item to exclude (recently produced items with free storage)

In [ ]:
# Type correct initial date for period being charged + verify number of weeks to charge
ini_date = '29-07-20'
ini_date_w1 = dt.datetime.strptime(ini_date, "%d-%m-%y")
end_date_w1 = ini_date_w1 + dt.timedelta(days=7)
end_date_w2 = ini_date_w1 + dt.timedelta(days=14)
end_date_w3 = ini_date_w1 + dt.timedelta(days=21)
end_date_w4 = ini_date_w1 + dt.timedelta(days=28)
end_date_w5 = ini_date_w1 + dt.timedelta(days=35)

end_date_all = [end_date_w1,end_date_w2,end_date_w3,end_date_w4,end_date_w5]

In [ ]:
excl_w1 = excl.loc[(excl['Transaction Date']>=ini_date_w1) & (excl['Transaction Date']<=end_date_w1),:]
excl_w2 = excl.loc[(excl['Transaction Date']>end_date_w1) & (excl['Transaction Date']<=end_date_w2),:]
excl_w3 = excl.loc[(excl['Transaction Date']>end_date_w2) & (excl['Transaction Date']<=end_date_w3),:]
excl_w4 = excl.loc[(excl['Transaction Date']>end_date_w3) & (excl['Transaction Date']<=end_date_w4),:]
excl_w5 = excl.loc[(excl['Transaction Date']>end_date_w4) & (excl['Transaction Date']<=end_date_w5),:]

excl_all = [excl_w1,excl_w2,excl_w3,excl_w4,excl_w5]

excl_all = pd.concat(excl_all, keys=['W1','W2','W3','W4','W5'])
excl_all = excl_all.reset_index().drop(columns='level_1').rename(columns={'level_0':'Week'})
excl_all = excl_all.groupby(['Week','Product Owner','Item Code'])[['QTY Completed']].sum().reset_index()

In [ ]:
excl_twe = (excl_all.loc[excl_all['Product Owner'] == 'CUS000404 Treasury Wine Estates']).copy()

Combine all relevant information into single dataframe

In [ ]:
csldt_twe = pd.merge(twe_w_all,excl_twe[['Week','Item Code','QTY Completed']],left_on=['Week','Item'], \
                     right_on=['Week','Item Code'],how='left').drop(columns='Item Code').fillna(0)
csldt_twe['Units Charged'] = csldt_twe['Ending Inv Qty On-hand'] - csldt_twe['QTY Completed']
csldt_twe['Pallets Charged'] = csldt_twe['Units Charged'].div(csldt_twe['Units per Pallet'], \
                                                                  fill_value=0).apply(np.ceil)

Produce output reports

In [ ]:
twe_summ_pallets = csldt_twe.groupby(['Week','Item Type: Long Name','Pallet Type'])['Pallets Charged'].sum()
twe_summ_pallets = twe_summ_pallets.unstack(0)

In [ ]:
twe_summ_units = csldt_twe.groupby(['Week','Item Type: Long Name','Pallet Type'])['Units Charged'].sum()
twe_summ_units = twe_summ_units.unstack(0)

In [ ]:
twe_exceptions_db = csldt_twe.loc[csldt_twe['Units per Pallet']<=0,:]
twe_exceptions_items = twe_exceptions_db.groupby(['Item'])[['Units Charged']].sum()

Export to Excel

In [ ]:
def summaries_single_sheet(reports, file_name):
    writer = pd.ExcelWriter(file_name)
    n=0
    row=1
    for report in reports:
        report.to_excel(writer, sheet_name = 'Single Sheet', startrow = row, startcol = 0)
        n+=1
        row=row+len(report)+5
    writer.save()
reps = [twe_summ_pallets,twe_summ_units,twe_exceptions_items]
summaries_single_sheet(reps, 'TWE Storage reports - 2020-09-01.xlsx')

In [ ]:
def twe_output_to_excel_multisheets(reports_list, sheets_list, file_name):
    writer = pd.ExcelWriter(file_name)
    n=0
    for report, sheet in zip(reports_list, sheets_list):
        report.to_excel(writer, sheet_name = sheet, startrow = 0, startcol = 0)
        n+=1
    writer.save()

reports = [csldt_twe,twe_exceptions_db]
sheets = ['TWE database','Exceptions database']

twe_output_to_excel_multisheets(reports, sheets, 'TWE Storage reports - 2020-09-01-2.xlsx')

##### Customers excluding TWE

In [ ]:
# Consolidating selected data into single dataframe

inv_sel = [w31,w32,w33,w34,w35]

for w in inv_sel:
    w['Stored Units'] = w[['Beginning Inv Qty On-hand','Ending Inv Qty On-hand']].min(axis=1)

stor_all = pd.concat(inv_sel, keys=['w31','w32','w33','w34','w35'], copy = True)\
            .reset_index().rename(columns={'level_0':'Week'}).drop(columns='level_1')

##### Units conversion and storage charging parameters (master data) preparation

In [ ]:
conv = stor_a['Master Data and Conversion']

In [ ]:
# List of Cost Categories that do not raise storage charges

excluded_items = ['Dry Goods- Labels','Raw Materials','Pallet', 'Wine Batch', 'Additives',\
            'Dry Goods- Others', 'Beverage Batch','Rework Item']
excluded_customers = ['CUS000284 Brix & Co Limited', 'CUS000424 Brix - Production', 'CUS000399 SKU Ltd']

In [ ]:
# Cleaning data in column used for conversion, so it can be of the float type (as opposed to object)
# NOTE: Used to clean data first time. Deactivated now because it will raise an error since there's no more str in the data

#conv['Units to Pallet'] = conv['Units to Pallet'].replace({'TBC':0,'':0}).astype(float)

In [ ]:
# Classifying items between chargeable and non chargeable ones for storage

# conv['Storage Chargeable'] = conv['Cost Category'].apply(lambda x: 'No' if x in(excluded_items) else 'Yes')

##### Base data preparation for period

In [ ]:
# Selecting tabs that will be used in calculation of storage charges

w31 = stor_a['Week 31'].copy()
w32 = stor_a['Week 32'].copy()
w33 = stor_a['Week 33'].copy()
w34 = stor_a['Week 34'].copy()
w35 = stor_a['Week 35'].copy()

In [ ]:
# Consolidating selected data into single dataframe

inv_sel = [w31,w32,w33,w34,w35]

for w in inv_sel:
    w['Stored Units'] = w[['Beginning Inv Qty On-hand','Ending Inv Qty On-hand']].min(axis=1)

stor_all = pd.concat(inv_sel, keys=['w31','w32','w33','w34','w35'], copy = True)\
            .reset_index().rename(columns={'level_0':'Week'}).drop(columns='level_1')

In [ ]:
# Adding conversion values from Conversion (master data table) into dataframe

stor_all = stor_all.merge(conv[['Name','Pallet Conversion Units','Domestic/Export', 'Cost Category','Chep/Non-Chep', 'Supplier','Storage Chargeable']], how='left', left_on='Item', right_on='Name')

# Calculating the number of pallets stored based on units stored and conversion parameters to pallets

stor_all['Pallets Stored'] = stor_all['Stored Units'].div(stor_all['Pallet Conversion Units'],fill_value=0)
stor_all = stor_all.replace([np.inf,-np.inf], np.nan)
stor_all_roundup = stor_all.copy()
stor_all_roundup['Pallets Stored'] = stor_all['Pallets Stored'].apply(np.ceil)

In [ ]:
weeks_gen = ['w31', 'w32', 'w33', 'w34', 'w35']
stor_gen = stor_all_roundup.loc[stor_all['Product Owner'] != 'CUS000404 Treasury Wine Estates']
stor_gen = stor_gen.loc[stor_gen['Week'].isin(weeks_gen)]

weeks_twe = ['w31', 'w32', 'w33', 'w34', 'w35']
stor_twe = stor_all_roundup.loc[stor_all['Product Owner'] == 'CUS000404 Treasury Wine Estates']
stor_twe = stor_twe.loc[stor_twe['Week'].isin(weeks_twe)]

##### Reference data verification (comparison) by Item Type (e.g. glass, cartons, caps)

In [ ]:
data_verif = stor_all_roundup.copy()

In [ ]:
data_verif = data_verif.loc[~data_verif['Cost Category'].isin(excluded_items)]
data_verif = data_verif.loc[~data_verif['Product Owner'].isin(excluded_customers)]

In [ ]:
data_verif = data_verif.groupby(['Cost Category', 'Item'])[['Stored Units','Pallet Conversion Units']].mean()

In [ ]:
data_verif['Check Parameters'] = np.where((data_verif['Stored Units'] >0) & (data_verif['Pallet Conversion Units'] == 0),'Verify',np.nan)

##### Summary report

In [ ]:
charge_summ_gen = stor_gen.loc[stor_gen['Storage Chargeable'] == 'Yes'].copy()

charge_summ_gen = charge_summ_gen.groupby(['Product Owner', 'Item Type: Long Name','Week'])\
            [['Pallets Stored']].sum().fillna(0).unstack([1,2])
charge_summ_gen['Total Pallets'] = charge_summ_gen['Pallets Stored'].sum(axis=1)

charge_summ_gen = charge_summ_gen.sort_values(by='Total Pallets', ascending=False)

In [ ]:
charge_summ_twe = stor_twe.loc[stor_all_roundup['Storage Chargeable'] == 'Yes'].copy()

charge_summ_twe = charge_summ_twe.groupby(['Chep/Non-Chep','Item Type: Long Name','Week'])\
            [['Pallets Stored']].sum().fillna(0).unstack([0,1])
charge_summ_twe['Total Pallets'] = charge_summ_twe['Pallets Stored'].sum(axis=1)

charge_summ_twe = charge_summ_twe.sort_values(by='Week')

In [ ]:
charge_detail = stor_all_roundup.loc[stor_all['Storage Chargeable'] == 'Yes'].copy()
charge_detail = charge_detail.groupby(['Product Owner', 'Item Type: Long Name','Item','Week'])\
            [['Stored Units','Pallets Stored']].sum().fillna(0).unstack(3)
#charge_detail['Total Pallets'] = charge_detail['Pallets Stored'].sum(axis=1)

In [ ]:
charge_summ_twe.loc['Total'] = charge_summ_twe.sum(axis=0)

##### Export to Excel

In [ ]:
def storage_output_to_excel_multisheets(reports_list, sheets_list, file_name):
    writer = pd.ExcelWriter(file_name)
    n=0
    for report, sheet in zip(reports_list, sheets_list):
        report.to_excel(writer, sheet_name = sheet, startrow = 0, startcol = 0)
        n+=1
    writer.save()

reports = [charge_summ_gen, charge_summ_twe, charge_detail, data_verif]
sheets = ['Summary Report General', 'Summary Report TWE','Detailed Report','Data Verification']

storage_output_to_excel_multisheets(reports, sheets, 'Storage Charges Weeks 31-35.xlsx')

# Useful snipets of code

In [ ]:
# Boolean indicator of values not equal to variable
c.ne(3)

In [ ]:
# Change values using IF statements or based on conditions

apx['Internal ID'] = apx['Entered to Due'].apply(lambda x: 'Lommen' if x==16 else 'Pakka')

apx.loc[apx['Entered to Due'] == 16, 'Internal ID'] = 'Penga'

In [ ]:
#produce an aggregation operation (e.g. sum) in same shape as original dataframe

total_order = a_fy2.groupby('ID')[['FY20']].transform('sum')

In [ ]:
# List duplicates in dataframe
a_fy2.loc[a_fy2['ID'].duplicated()]

In [ ]:
# Segregate dataframe into two by defined criteria
b1 = b[b['Market']=='China']
b2 = b.drop(b1.index)

In [ ]:
from platform import python_version
python_version()

In [ ]:
#get folder from which python is being executed
sys.executable

In [ ]:
# ! df.index.sortlevel(level=[0,1], ascending=[True,False])

In [ ]:
# Using date of TODAY
#ap1a = ap1[(ap1['Status']=='Open') & (ap1['Due Date/Receive By']>dt.datetime.today())]

In [ ]:
# CSV import - replacing undesired characters
# ! manuf_cost1['TOTAL'] = manuf_cost1['TOTAL'].str.replace(',','')
# ! manuf_cost1['TOTAL'] = manuf_cost1['TOTAL'].astype('float')

In [ ]:
# Select columns by type
production_hours = volumes.select_dtypes(exclude=['object','datetime']).div(machine_output).combine_first(volumes)[volumes.columns]

In [ ]:
# Renaming columns with values from a row of data
c1.columns = c1.iloc[0]

In [ ]:
# Combining text with data output
for i in range(0,4):
    print('Revenue FY2'+str(i)+' is '+ str(b[b.FY == 'FY2'+str(i)].Revenue.sum()) + \
          ' and Volume is ' + str(b[b.FY == 'FY2'+str(i)].Units.sum()))

In [ ]:
# ! Order by categorical data

# When index

order = ['In Negotiations', 'Existing', 'In Discussion']
b.set_index(['Status'])
b.reindex(order)

# When column

order = ['In Negotiations', 'Existing', 'In Discussion'] # these are values already in the 'Status' field
b['Status'] = pd.Categorical(b['Status'], categories = order)
b=b.sort_values(by='Status')
b


In [ ]:
# ! Duplicate / repeat lines of dataframe

# using np.repeat
newdf = pd.DataFrame(np.repeat(df.values,3,axis=0)) #3 is number of times to repeat
newdf.columns = df.columns

# using pd.concat
dic = [c,c,c]
d = pd.concat(dic, axis=0).reset_index(drop=True)

In [ ]:
# Styling

"""
two_dec_list = ['V19 COGS', 'V19 Price']
# set ALL float columns to '${:,.2f}' formatting (including the percentage)
format_dict = {col_name: '${:,.2f}' for col_name in two_dec_list}
# override the percentage column
format_dict['Units'] = '{:,.0f}'
format_dict['Revenue'] = '${:,.0f}'
format_dict['COGS'] = '${:,.0f}'
format_dict['Margin'] = '${:,.0f}'
#format_dict['Period'] = '{%b-%y}'

onsite = onsite.style.format(format_dict)
"""

Experimenting with styling and formating

In [ ]:
total['Margin %']=(total['Revenue']/total['COGS']-1).astype(float)

In [ ]:
total_styled = total.style\
    .apply(lambda x: ['background:red' if x < 500 else 'background:green' for x in total.Revenue], axis=0)\
    .applymap(lambda x: 'background-color: %s' % 'red' if x <10 else 'background-color: %s' % 'green', subset=['V19 Price','V19 COGS'])\
    .applymap(lambda x: 'background-color: %s' % 'red' if x <1000 else 'background-color: %s' % 'green', subset=['Revenue'])\
    .format({'V19 COGS': '${:20,.2f}'})\
    .format({'V19 Price': '${:20,.2f}'})\
    .format({'Margin %': '{:.2%}'})\
    .format({'Period': '{:%d-%b-%y}'})

In [ ]:
total_styled

In [ ]:
rep_styled=op.load_workbook('Beverage Sales Report.xlsx')

In [ ]:
ws=rep_styled['Monthly (v1)']

In [ ]:
# Import necessary style classes
from openpyxl.styles import Font, Color, Alignment, Border, Side, colors

from openpyxl.styles import NamedStyle

# Let's create a style template for the header row
header = NamedStyle(name="header2")
header.font = Font(bold=True)
header.border = Border(bottom=Side(border_style="thin"))
header.alignment = Alignment(horizontal="center", vertical="center")

# Now let's apply this to all first row (header) cells
header_row = ws[2]
for cell in header_row:
    cell.style = header

rep_styled.save(filename="Styled Bev Sales Reports.xlsx")

# Real Python Guide to Working with Excel

In [ ]:
Brand=['Honda','Nissan','Triumph','Holden']
Model=['CRV','Leaf','Tiger','Barina']
Type=['Car','Car','Moto','Car']
Wheels={'Car':4,'Moto':2}
Price=[23000,43000,23000,9000]

columns=['Brand','Model','Type','Price']
a=zip(Brand,Model,Type,Price)

b=pd.DataFrame(a, columns=columns)

b['Wheels']=b['Type'].map(Wheels)

In [ ]:
b

In [ ]:
c = b

In [ ]:
c

In [ ]:
dic = [c,c,c]
d = pd.concat(dic, axis=0).reset_index(drop=True)

In [ ]:
b[b.Brand.str.contains('ss|de')]

#### Working with Excel

RealPython - A Guide to Excel Spreadsheets in Python With openpyxl - Amazon database example

In [ ]:
wb=op.load_workbook('reviews-sample.xlsx')

In [ ]:
ws=wb.active

In [ ]:
ws['A1'].value

In [ ]:
for value in ws.iter_rows(1,1,values_only=True):
    print(value)

In [ ]:
for value in ws.iter_rows(2,None,4,7, values_only=True):
    print(value)

In [ ]:
import json

In [ ]:
products = {}

for row in ws.iter_rows(2,None,4,7, values_only=True):
    product_id=row[0]
    product = {'parent': row[1],
              'title': row[2],
              'category': row[3]}
    products[product_id] = product

print(json.dumps(products))

In [ ]:
for cell in ws[1]:
    print(cell.value)

In [ ]:
from openpyxl import load_workbook
from datetime import datetime
from classes import Product, Review
from mapping import PRODUCT_ID, PRODUCT_PARENT, PRODUCT_TITLE, PRODUCT_CATEGORY, REVIEW_DATE, REVIEW_ID, \
                    REVIEW_CUSTOMER, REVIEW_STARS, REVIEW_HEADLINE, REVIEW_BODY

wb = load_workbook(filename='reviews-sample.xlsx', read_only=True)
ws = wb.active

products = []
reviews = []

for row in ws.iter_rows(min_row=2, values_only=True):
    product = Product(id=row[PRODUCT_ID], parent=row[PRODUCT_PARENT], title=row[PRODUCT_TITLE], category=row[PRODUCT_CATEGORY] )
    products.append(product)

    spread_date = row[REVIEW_DATE]
    parsed_date = datetime.strptime(spread_date, '%Y-%m-%d')
    
    review = Review(id=row[REVIEW_ID], customer_id=row[REVIEW_CUSTOMER],
                    stars=row[REVIEW_STARS],
                    headline=row[REVIEW_HEADLINE],
                    body=row[REVIEW_BODY],
                    date=parsed_date)
    reviews.append(review)

print(products[0])
print(reviews[0])


#### Working with Excel

pd.ExcelFile # Loads Excel file from Pandas

In [ ]:
xl1 = pd.ExcelFile('Beverage Sales FY21-23 Projection v5.xlsx')

In [ ]:
xl1

In [ ]:
xl1.sheet_names

In [ ]:
for sheet in xl1.sheet_names:
    wb.active = wb[sheet]
    wb[sheet]['A1'] = 'Johnny'
wb.save('Beverage Sales FY21-23 Projection v5 out.xlsx')

In [ ]:
dfxl1 = xl1.parse('Monthly')

op.load_workbook # loads Excel file with Openpyxl

In [ ]:
wb=op.load_workbook('Beverage Sales FY21-23 Projection v5.xlsx')

In [ ]:
wb.worksheets

In [ ]:
wb.sheetnames

In [ ]:
for sheet in wb.sheetnames:
    wb.active = wb[sheet]
    wb[sheet]['A1'] = 'Johnny'
wb.save('Beverage Sales FY21-23 Projection v5 out.xlsx')

In [ ]:
wb.active = wb['Monthly']

In [ ]:
wb.active

pd.read_excel # Loads Excel selected worksheets as dataframes or dictionaries if several worksheets

In [ ]:
xl2 = pd.read_excel('Beverage Sales FY21-23 Projection v5.xlsx', sheet_name = None)

In [ ]:
xl2.keys()

In [ ]:
xl2.values()

In [ ]:
dfxl2 = xl2['Sales Fcst']

In [ ]:
l = list(xl2.keys())

In [ ]:
for sheet in l:
    xl2[sheet].iloc[1,1] = 'Johnny'

In [ ]:
xl2['Sales Fcst']

Iterating across cells in Excel

In [ ]:
for value in ws.iter_rows(1,1,values_only=True):
    print(value)

In [ ]:
for value in ws.iter_rows(2,None,4,7, values_only=True):
    print(value)

Retrievinga and entering values in cells in Excel

In [ ]:
ws['A1'].value

In [ ]:
ws['A1'].value = 34

Entering a formula in Excel

In [ ]:
ws['P1'] = '=AVERAGE(H2:H100)'

# Member Dues Email (exercise)

In [ ]:
ws['A1'].valueimport openpyxl as op
import smtplib as s
import sys

In [ ]:
wb = op.load_workbook('duesRecords.xlsx')
sheet = wb['Sheet1']

In [ ]:
lastCol = sheet.max_column
latestMonth = sheet.cell(row=1, column=lastCol).value

In [ ]:
unpaidMembers = {}
for r in range(2, sheet.max_row + 1):
    payment = sheet.cell(row=r, column=lastCol).value
    if payment != 'paid':
        name = sheet.cell(row=r, column=1).value
        email = sheet.cell(row=r, column=2).value
        unpaidMembers[name] = email

In [ ]:
unpaidMembers

In [ ]:
email = s.SMTP('smtp.office365.com', 587)
email.ehlo()
email.starttls()
email.login('joao.melo@brix.co.nz', 'TheL0dg3!!')



In [ ]:
for name, email in unpaidMembers.items():
    body = "Subject: %s dues unpaid. \nDear %s,\nRecords show that you have not paid dues for %s. Please make this payment as soon as possible. Thank you!" %(latestMonth, name, latestMonth)
    print('Sending email to %s...' % email)

In [ ]:
email.quit()

# Short tests

In [ ]:
os.chdir(r'C:\Users\joao.melo\PycharmProjects\LearnPython')

In [ ]:
a=op.load_workbook('Cash Flow reports.xlsx')
s = a.active

In [ ]:
for i in range(0,30):
    s.row_dimensions[i+1].height=50
a.save('Cash Flow Rerports Wide Rows.xlsx')

In [ ]:
for row in s.iter_rows(2,20,None,None):
    s.row_dimensions.height=50
a.save('Cash Flow Rerports Wide Rows.xlsx')